# 랭체인 멀티턴

- LangChain의 RunnableWithMessageHistory를 사용하여 대화기록을 관리하는 챗봇 구현
- 세션 ID별로 대화 기록이 어떻게 분리되어 저장되는지 확인

## 라이브러리 불러오기

In [1]:
from langchain_core.chat_history import InMemoryChatMessageHistory  # 메모리에 대화 기록을 저장하는 클래스
from langchain_core.runnables.history import RunnableWithMessageHistory  # 메시지 기록을 활용해 실행 가능한 래퍼wrapper 클래스
from langchain_openai import ChatOpenAI  # 오픈AI 모델을 사용하는 랭체인 챗봇 클래스
from langchain_core.messages import HumanMessage

## API KEY 불러오기

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os

In [3]:
load_dotenv()

True

## 대화 기록 함수 정의

In [4]:
model = ChatOpenAI(model="gpt-4o-mini")

# 세션별 대화 기록을 저장할 딕셔너리
store = {}

# 세션 ID에 따라 대화 기록을 가져오는 함수
def get_session_history(session_id: str):
    # 만약 해당 세션 ID가 store에 없으면, 새로 생성해 추가함
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # 메모리에 대화 기록을 저장하는 객체 생성
    return store[session_id]  # 해당 세션의 대화 기록을 반환

# 모델 실행 시 대화 기록을 함께 전달하는 래퍼 객체 생성
with_message_history = RunnableWithMessageHistory(model, get_session_history)

## 첫 번째 대화

In [5]:
config = {"configurable": {"session_id": "abc2"}}  # 세션 ID를 설정하는 config 객체 생성

response = with_message_history.invoke(
    [HumanMessage(content="안녕? 난 김정은이야.")],
    config=config,
)

print(response.content)

안녕하세요, 김정은님! 어떻게 도와드릴까요?


## 두 번째 대화

In [6]:
response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

print(response.content)

당신의 이름은 김정은입니다. 다른 질문이나 도움이 필요하시면 말씀해 주세요!


## 세션 테스트

In [7]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="내 이름이 뭐지?")],
    config=config,
)

response.content

'죄송하지만, 당신의 이름을 알 수 없습니다. 이름을 말씀해 주시면 반갑게 인사해 드릴 수 있습니다!'

In [8]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="아까 우리가 무슨 얘기 했지?")],
    config=config,
)

response.content

'우리는 당신의 이름이 김정은이라는 것에 대해 이야기했어요. 추가로 논의하고 싶은 주제나 질문이 있다면 말씀해 주세요!'

## 스트리밍 응답 출력

In [9]:
config = {"configurable": {"session_id": "abc2"}}
for r in with_message_history.stream(
    [HumanMessage(content = "내가 어느 나라 사람인지 맞춰보고, 그 나라의 문화에 대해 말해봐")],
    config=config,
):
    print(r.content, end="")
    # print(r.content, end="|")

당신의 이름이 김정은인 점을 고려했을 때, 한국 사람일 가능성이 높습니다. 

한국의 문화에 대해 간단히 말씀드리면:

1. **음식**: 한국의 음식은 김치, 불고기, 비빔밥 등이 유명합니다. 한국 음식은 보통 매운 맛과 다양한 재료가 특징입니다.

2. **전통 의상**: 한복은 한국의 전통 의상으로, 특별한 날이나 명절에 입습니다. 다채로운 색상과 독특한 디자인이 특징입니다.

3. **명절**: 설날(설날)과 추석(한가위)은 한국에서 가장 중요한 명절로, 가족들이 모여 음식을 나누고 조상을 기리는 시간을 갖습니다.

4. **한글**: 한국의 공식 문자 시스템인 한글은 15세기 세종대왕에 의해 만들어졌습니다. 한글은 배우기 쉽고 효율적인 문자로 알려져 있습니다.

5. **K-드라마 및 K-Pop**: 최근 몇 년간 K-드라마와 K-Pop이 전 세계적으로 유명해졌으며, 많은 사람들이 한국의 대중문화에 관심을 가지고 있습니다.

더 궁금한 점이 있으면 언제든지 말씀해주세요!